<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, Point
import folium
from geopy.distance import geodesic

In [12]:
# 'trips.xlsx' is uploaded to Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition
1954,77403 (М947ОК67) Renault Logan,17.02.2025 23:49:43,"55,73905°, 37,817688°",11,+
2003,77403 (М947ОК67) Renault Logan,17.02.2025 23:56:31,"55,767463°, 37,833638°",28,+


In [13]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips['Сoordinates'] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
df_trips[['Longitude', 'Latitude']] = df_trips['Сoordinates'].str.split('. ', expand=True)
# df_trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

# df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

# Преобразование поездок в геометрию
df_trips['geometry'] = [Point(lon, lat) for lon, lat in zip(df_trips['Longitude'], df_trips['Latitude'])]
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry
1363,77403 (М947ОК67) Renault Logan,2025-02-17 18:26:09,55.76686. 37.830355,5,+,55.76686,37.830355,POINT (55.76686 37.83036)
3103,77403 (М947ОК67) Renault Logan,2025-02-18 17:30:43,55.754498. 37.817791,0,+,55.754498,37.817791,POINT (55.7545 37.81779)


In [14]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")

# в файле bus-lanes.geojson долгота-широта вместо широта-долгота
# Меняем порядок координат в каждой геометрии
# def swap_coords(geometry):
#     if geometry.geom_type == "LineString":
#         return LineString([(lat, lon) for lon, lat in geometry.coords])
#     elif geometry.geom_type == "MultiLineString":
#         return geometry.__class__([LineString([(lat, lon) for lon, lat in line.coords]) for line in geometry.geoms])
#     return geometry  # На случай других типов геометрии

# gdf_bus_lanes["geometry"] = gdf_bus_lanes["geometry"].apply(swap_coords)

# Устанавливаем корректную CRS
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")
gdf_bus_lanes[gdf_bus_lanes['short_name'] == 'Москва'].sample(2)

,admin_level,index_right,lanes,length,name,place,population,short_name,type,lanes_length,geometry
573,None,NaN,2,1801.676561,Москва,None,12655050.0,Москва,None,3603.353123,"LINESTRING (37.57597 55.51525, 37.57125 55.49935)"
736,None,NaN,1,134.005068,Москва,None,12655050.0,Москва,None,134.005068,"LINESTRING (37.75639 55.75933, 37.75439 55.75891)"


In [15]:
# Преобразуем CRS в метры (EPSG:3857)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)
# Делаем buffer на 30 метров
gdf_bus_lanes["geometry"] = gdf_bus_lanes.geometry.buffer(30)
# Возвращаем обратно в географические координаты (EPSG:4326)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)
gdf_bus_lanes.sample(2)

,admin_level,index_right,lanes,length,name,place,population,short_name,type,lanes_length,geometry
623,None,NaN,1,1979.753182,Москва,None,12655050.0,Москва,None,1979.753182,"POLYGON ((37.62265 55.72379, 37.62269 55.71176..."
366,6,594.0,1,519.977736,городской округ Иркутск,None,617515.0,Иркутск,None,519.977736,"POLYGON ((104.20385 52.3411, 104.20387 52.3411..."


In [18]:
gdf_bus_lanes_M = gdf_bus_lanes[gdf_bus_lanes['short_name'] == 'Москва']
gdf_bus_lanes_M.sample(2)

,admin_level,index_right,lanes,length,name,place,population,short_name,type,lanes_length,geometry
534,None,NaN,2,1806.931608,Москва,None,12655050.0,Москва,None,3613.863217,"POLYGON ((37.80219 55.7159, 37.80222 55.71589,..."
556,None,NaN,2,216.887182,Москва,None,12655050.0,Москва,None,433.774365,"POLYGON ((37.58217 55.75969, 37.58355 55.75939..."


In [19]:
# Проверяем пересечение точек с расширенными автобусными полосами
df_trips["on_bus_lane"] = df_trips["geometry"].apply(lambda point: gdf_bus_lanes_M.geometry.union_all().intersects(point))

# Вывод случайных 2 строк для проверки
df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane
301,77403 (М947ОК67) Renault Logan,2025-02-17 07:02:02,55.772098. 37.820046,36,+,55.772098,37.820046,POINT (55.7721 37.82005),False
1961,77403 (М947ОК67) Renault Logan,2025-02-17 23:50:18,55.742805. 37.81562,58,+,55.742805,37.81562,POINT (55.7428 37.81562),False


In [20]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)

# Вычисляем расстояние между последовательными точками в км
# Создаем два массива для координат
coords1 = np.column_stack((df_trips['Latitude'], df_trips['Longitude']))
coords2 = np.column_stack((df_trips['next_lat'], df_trips['next_lon']))

# Функция для вычисления расстояний
def calculate_distances(coords1, coords2):
    return np.array([geodesic((lat1, lon1), (lat2, lon2)).meters / 1000
                     if not pd.isna(lat2) else np.nan
                     for (lat1, lon1), (lat2, lon2) in zip(coords1, coords2)])

# Применяем функцию для всей таблицы
df_trips['distance'] = calculate_distances(coords1, coords2)



# Вычисляем время между последовательными точками, часов, Заменяем NaN на None
time_diff = (df_trips["next_time"] - df_trips['Day_time']).dt.total_seconds() / 3600
df_trips["time"] = time_diff.where(df_trips["next_time"].notna(), None)



# Проверяем, чтобы значения в столбцах "distance" и "time" были валидными, и вычисляем скорость
df_trips["speed"] = np.where(
    (pd.notnull(df_trips["distance"])) & (df_trips["time"].notnull()) & (df_trips["time"] != 0),
    df_trips["distance"] / df_trips["time"],
    None
)

In [ ]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875


In [21]:
# Оценка скорости на выделенных полосах
avg_speed_bus_lane = 50

# Рассчитываем предсказанное время для каждой строки
df_trips['predicted_time'] = np.where(
    df_trips["on_bus_lane"],
    np.minimum(df_trips['time'], df_trips['distance'] / avg_speed_bus_lane),
    df_trips['time']
)

In [22]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed,predicted_time
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,+,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852,0.001111
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,+,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727,0.000278
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,+,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875,0.000278


In [23]:
# время в движении, часов
total_time = df_trips.loc[df_trips['speed'] > 0, 'time'].sum()

# Выигрыш во времени
df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time :.2f} часов")
print(f"% экономии времени: {total_time_saved / total_time  * 100  :.2f} %")

Общая экономия времени: 6.98 часов
% экономии времени: 0.00 %
